## 1. Import libraries 

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor


## 2. Download datasets 

In [ ]:
# Generate a dictionary to read the csv using float32

def Convert(a):
    it = iter(a)
    res_dct = dict(zip(it, it))
    return res_dct



feature_list = []
for i in range(300):
    feature_list.append('f_{}'.format(i))
    feature_list.append('float32') # comment this line to get a list with the name of the features
        
feature_dict = Convert(feature_list)

In [ ]:
df_train = pd.read_csv('../input/ubiquant-market-prediction/train.csv', dtype = feature_dict, nrows = 1100000)

df_train.head()


In [ ]:
df_train = df_train.drop('row_id', axis = 1)
df_train.tail()

In [ ]:
# import time
# from datetime import datetime
# totimestamp = lambda s: np.int32(time.mktime(datetime.strptime(s, "%d/%m/%Y").timetuple()))
# df_train =df_train.set_index('timestamp')
# df_train = df_train.loc[totimestamp("15/05/2021") : totimestamp("20/09/2021")]
# df_train = df_train.reset_index()

In [ ]:
# df_asset_details = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/asset_details.csv')
# df_asset_details

In [ ]:
# def get_features(data):
#     # FE for data as row of DataFrame
    
#     # Two new features from the competition tutorial
#     df_feat = data[['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP']].copy()
#     df_feat['Upper_Shadow'] = df_feat['High'] - np.maximum(df_feat['Close'], df_feat['Open'])
#     df_feat['Lower_Shadow'] = np.minimum(df_feat['Close'], df_feat['Open']) - df_feat['Low']
    
#     # Thanks to https://www.kaggle.com/code1110/gresearch-simple-lgb-starter
#     df_feat['high2low'] = df_feat['High'] / df_feat['Low']
#     df_feat['volume2count'] = df_feat['Volume'] / (df_feat['Count'] + 1)
    
#     return df_feat

In [ ]:
def get_data_for_investment(df_train, asset_id):
    # Get X and y
    
    df_proc = df_train[df_train["investment_id"] == asset_id]
    X = df_proc.drop(['target','time_id'], axis=1)
    y = df_proc["target"]
    
    return X, y

## 3. Model training

In [ ]:
def model_training(X,y):
    # Model training
    
    model = LGBMRegressor(n_estimators=5000,num_leaves=700,learning_rate=0.1)
    model.fit(X, y)
    
    return model

In [ ]:
%%time
Xs = {}
ys = {}
models = {}

#for investment_id in df_train['investment_id'].unique():
for investment_id in [1,2]:
    print("Training model for (Investment_id= {})".format(investment_id))
    X, y = get_data_for_investment(df_train, investment_id)    
    model = model_training(X,y)
    Xs[investment_id], ys[investment_id], models[investment_id] = X, y, model

In [ ]:
# Check the model and it's possibility for the prediction 
print("Check the model and it's possibility for the prediction")
x = df_train.drop(['target','time_id'], axis = 1).iloc[2]
y_pred = models[1].predict([x])
y_pred[0]

In [ ]:
#df_train.drop(['target','time_id'], axis = 1).iloc[0]

In [ ]:
%%time
# model 2 - Use this model for the unobservable investments
X2 = df_train.drop(['target','time_id','investment_id'], axis=1)
y2 = df_train['target']

model2 = model_training(X2,y2)

In [ ]:
model2

In [ ]:
# Check the model and it's possibility for the prediction 
print("Check the model 2 and it's possibility for the prediction")
x = df_train.drop(['target','time_id','investment_id'], axis = 1).iloc[0]
y_pred = model2.predict([df_train.drop(['target','time_id','investment_id'], axis = 1).iloc[0]])
y_pred[0]

In [ ]:
# # recover memmory
# df_train = None

In [ ]:

# test_models = np.load('models.npy',allow_pickle='TRUE').item()

## Submission

In [ ]:
import ubiquant
env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
for test_df, sample_prediction_df in iter_test:
    for i in range(len(test_df)):
        if test_df['investment_id'][i] in models:
            y_pred = models[test_df['investment_id'][i]].predict([test_df.drop('row_id', axis = 1).iloc[i]])
            sample_prediction_df.at[i, 'target'] = y_pred[0]
        else:
            y_pred = model2.predict([test_df.drop(['row_id','investment_id'], axis = 1).iloc[i]])
            sample_prediction_df.at[i, 'target'] = y_pred[0]
    print(sample_prediction_df)
    env.predict(sample_prediction_df)   # register your predictions